In [ ]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 14.4 MB/s eta 0:00:00


In [ ]:
!playwright install


163.5 MiB [] 0% 0.0s163.5 MiB [] 0% 9.1s163.5 MiB [] 0% 4.8s163.5 MiB [] 1% 4.1s163.5 MiB [] 1% 3.7s163.5 MiB [] 2% 3.4s163.5 MiB [] 3% 3.3s163.5 MiB [] 3% 3.2s163.5 MiB [] 4% 3.1s163.5 MiB [] 5% 2.9s163.5 MiB [] 6% 2.8s163.5 MiB [] 6% 3.2s163.5 MiB [] 7% 3.2s163.5 MiB [] 7% 3.3s163.5 MiB [] 8% 3.2s163.5 MiB [] 8% 3.1s163.5 MiB [] 9% 3.0s163.5 MiB [] 10% 3.0s163.5 MiB [] 11% 3.0s163.5 MiB [] 11% 2.9s163.5 MiB [] 12% 2.8s163.5 MiB [] 13% 2.8s163.5 MiB [] 13% 2.7s163.5 MiB [] 14% 2.7s163.5 MiB [] 14% 2.8s163.5 MiB [] 15% 2.8s163.5 MiB [] 16% 2.7s163.5 MiB [] 17% 2.6s163.5 MiB [] 18% 2.5s163.5 MiB [] 19% 2.4s163.5 MiB [] 20% 2.4s163.5 MiB [] 21% 2.3s163.5 MiB [] 22% 2.3s163.5 MiB [] 23% 2.2s163.5 MiB [] 24% 2.2s163.5 MiB [] 25% 2.1s163.5 MiB [] 26% 2.1s163.5 MiB [] 27% 2.0s163.5 MiB [] 28% 2.0s163.5 MiB [] 28% 2.1s163.5 MiB [] 29% 2.1s163.5 MiB [] 30% 2.0s163.5 MiB [] 31% 2.0s163.5 MiB [] 32% 1.9s163.5 MiB [] 33% 1.8s163.5 MiB [] 34% 1.8s163.5 MiB [] 35% 1.7s163.5 MiB [] 36% 1.7s163.5 MiB

In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import json
from bs4 import BeautifulSoup
import subprocess

# Jupyter Notebook에서 실행 가능하도록 설정
nest_asyncio.apply()

# Playwright 브라우저 설치 (필요 시 자동 설치)
try:
    subprocess.run(["playwright", "install"], check=True)
except subprocess.CalledProcessError:
    print("Playwright 브라우저 설치 실패. 수동으로 'playwright install'을 실행하세요.")

async def fetch_yju_scholarship_notices():
    base_url = "https://www.yju.ac.kr/kr/3261/subview.do?enc=Zm5jdDF8QEB8...&page={}"  # 페이지 번호로 접근
    notices = []
    page_numbers = range(1, 4)  # 최대 3페이지까지 탐색

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        for page_num in page_numbers:
            list_page_url = base_url.format(page_num)
            await page.goto(list_page_url, wait_until="networkidle")

            soup = BeautifulSoup(await page.content(), "html.parser")

            # 공지사항 제목 및 링크 추출
            notice_items = soup.select(".board-list .td-subject a")
            notice_dates = soup.select(".board-list .td-date")

            if not notice_items:
                break

            for item, date_tag in zip(notice_items, notice_dates):
                title = item.text.strip()
                date_str = date_tag.text.strip()
                detail_url = "https://www.yju.ac.kr" + item["href"]  # 상세페이지 URL 조합

                await page.goto(detail_url, wait_until="networkidle")
                final_soup = BeautifulSoup(await page.content(), "html.parser")

                content_div = final_soup.select_one(".board-view .board-content")
                content = content_div.get_text("\n", strip=True) if content_div else "본문 없음"

                if "장학" in title.lower() or "장학" in content.lower():
                    notices.append({
                        "title": title,
                        "url": detail_url,
                        "date": date_str,
                        "content": content
                    })

                await page.go_back()

        await browser.close()

    return notices

# 크롤링 실행
notices_data = asyncio.run(fetch_yju_scholarship_notices())
print(f"총 {len(notices_data)}개의 장학 공지사항을 크롤링했습니다.\n")

# 크롤링된 데이터 출력 (최대 5개 미리보기)
for i, notice in enumerate(notices_data[:5], 1):
    content_preview = notice['content'][:300]
    print(f"[{i}] {notice['title']} ({notice['date']})")
    print(f"   {notice['url']}")
    print(f"   {content_preview}...\n")


총 0개의 장학 공지사항을 크롤링했습니다.



In [ ]:
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright
import json
from bs4 import BeautifulSoup

# Jupyter Notebook에서 실행 가능하도록 설정
nest_asyncio.apply()

async def fetch_yju_scholarship_notices():
    base_url = "https://www.yju.ac.kr/kr/3261/subview.do"  # 페이지 번호로 접근
    notices = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # ✅ 디버깅을 위해 headless=False로 변경
        context = await browser.new_context()

        # ✅ 공지사항 목록 페이지에서 세션 유지
        page = await context.new_page()
        await page.goto(base_url, wait_until="networkidle")
        print('⏳ 기본 페이지 로딩 완료')

        await page.goto(base_url, wait_until="networkidle")
        await page.wait_for_load_state("domcontentloaded")
        print(f'🔗 페이지 이동 완료: {base_url}')

        soup = BeautifulSoup(await page.content(), "html.parser")

        # 🔹 공지사항 제목 및 상세 URL 추출
        notice_items = await page.locator("html > body > div:nth-of-type(1) > div:nth-of-type(3) > div:nth-of-type(2) > div:nth-of-type(2) > div:nth-of-type(3) > div > article > div > div > div > div:nth-of-type(2) > div > div > div > div > div > div:nth-of-type(2) > div:nth-of-type(1) > form:nth-of-type(2) > div > ul > li:nth-of-type(1) > ol > li:nth-of-type(1) > a > div > span").all()
        notice_dates = soup.select("tr td:nth-child(3)")

        if not notice_items:
            print("❌ 공지사항 제목을 찾을 수 없습니다. HTML 일부 출력:")
            html_content = await page.content()
            print(html_content[:2000])  # HTML 일부 출력
            return notices

        for idx, (item, date_tag) in enumerate(zip(notice_items, notice_dates)):
            title = item.text.strip()
            date_str = date_tag.text.strip()

            # ✅ 링크를 직접 방문하는 것이 아니라 클릭 방식으로 이동하도록 설정
            element_handle = await page.query_selector(f'html>body>div[1]>div[3]>div[2]>div[2]>div[3]>div>article>div>div>div>div[2]>div>div>div>div>div>div[2]>div[1]>form[2]>div>ul>li[1]>ol>li[{idx+1}]>a>div>span')
            if element_handle:
                print(f'✅ 링크 요소 발견 (공지 {idx+1}): {title}')
                print(f'🔹 클릭 시도: {title}')
                await element_handle.click()
                await page.wait_for_load_state("networkidle")  # 상세 페이지 로딩 대기
                print(f'⏳ 상세 페이지 로딩 완료: {title}')
                print(f'🖱️ 클릭 완료: {title}')  # ✅ 클릭 방식으로 이동

                # ✅ 페이지 로드 완료까지 대기
                await page.wait_for_load_state("networkidle")
                print('⏳ 상세 페이지 로딩 완료')

                # ✅ 리디렉션 감지: 만약 index.jsp로 이동하면 다시 시도
                if "index.jsp" in page.url:
                    print(f"⚠️ {title} 페이지가 index.jsp로 리디렉션됨. 크롤링 불가")
                    await page.go_back()
                print('🔙 목록 페이지로 돌아가기')

                final_soup = BeautifulSoup(await page.content(), "html.parser")

                # 🔹 상세 내용 크롤링
                content_div = final_soup.select("html>body>div[1]>div[3]>div[2]>div[2]>div[3]>div>article>div>div>div>div[2]>div>div>div>div>div>div>div>div>ul[2]>li *")

                if content_div:
                    content = content_div.get_text("\n", strip=True)
                else:
                    content = "본문 없음"

                # 🔹 "장학"이 포함된 공지만 저장
                if "장학" in title.lower() or "장학" in content.lower():
                    notices.append({
                        "title": title,
                        "url": page.url,  # ✅ 현재 페이지 URL 저장
                        "date": date_str,
                        "content": content
                    })

                await page.go_back()  # ✅ 목록 페이지로 다시 돌아가기

        await browser.close()

    return notices

# 📌 크롤링 실행
notices_data = asyncio.run(fetch_yju_scholarship_notices())
print(f"총 {len(notices_data)}개의 장학 공지사항을 크롤링했습니다.\n")

# ✅ 크롤링된 데이터 출력 (최대 5개 미리보기)
for i, notice in enumerate(notices_data[:5], 1):
    content_preview = notice['content'][:300]  # 본문 앞 300자만 출력
    print(f"🔹 [{i}] {notice['title']} ({notice['date']})")
    print(f"   👉 {notice['url']}")
    print(f"   📄 {content_preview}...\n")


⏳ 기본 페이지 로딩 완료
🔗 페이지 이동 완료: https://www.yju.ac.kr/kr/3261/subview.do
총 0개의 장학 공지사항을 크롤링했습니다.

